In [138]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
from sklearn.linear_model import LinearRegression

#Downloading stock price data from yfinance
warnings.filterwarnings('ignore')
tax_rate=0.25
tickers = ['^BSESN','KOTAKBANK.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'AXISBANK.NS' ,'INDUSINDBK.NS']
start_date = "2021-03-01"
end_date = "2025-3-31"
data = yf.download(tickers, start=start_date, end=end_date,threads=False,interval='1mo',auto_adjust=False)
print(data)

[*********************100%***********************]  6 of 6 completed

Price         Adj Close                                                       \
Ticker      AXISBANK.NS  HDFCBANK.NS ICICIBANK.NS INDUSINDBK.NS KOTAKBANK.NS   
Date                                                                           
2021-03-01   694.424377  1433.872070   562.691895    921.716248  1747.719604   
2021-04-01   711.798645  1355.777832   580.478516    902.885071  1743.532104   
2021-05-01   747.443298  1455.183594   640.652954    978.258240  1802.254761   
2021-06-01   745.053711  1437.951904   609.864990    981.493286  1700.711670   
2021-07-01   705.824646  1375.287964   659.744507    947.355652  1649.815552   
2021-08-01   783.088013  1524.680420   697.111023    958.364746  1748.417358   
2021-09-01   763.224609  1537.744507   679.466431   1079.061157  2000.451782   
2021-10-01   738.781006  1526.078491   777.578613   1106.525269  2026.037354   
2021-11-01   652.805725  1439.981445   692.554443    856.921387  1956.961670   
2021-12-01   675.606262  1426.338989   7

In [139]:
#Downloading balance sheets for each company
balancesheets={}
for t in tickers[1:]:
    stock = yf.Ticker(t)
    bs = stock.balance_sheet
    balancesheets[t]=bs

In [140]:
#Calculating total debt and equity
debt_vals={}
equity_vals={}
debt_to_equity={}
for t in tickers[1:]:
    debt_vals[t]=balancesheets[t]['2024-03-31']['Total Debt']
    equity_vals[t]=balancesheets[t]['2024-03-31']['Stockholders Equity']
    debt_to_equity[t]=debt_vals[t]/equity_vals[t]

In [141]:
print(debt_vals)

{'KOTAKBANK.NS': 751056062000.0, 'HDFCBANK.NS': 8093552100000.0, 'ICICIBANK.NS': 2009987053000.0, 'AXISBANK.NS': 2281995500000.0, 'INDUSINDBK.NS': 476114113000.0}


In [142]:
print(equity_vals)

{'KOTAKBANK.NS': 1299716900000.0, 'HDFCBANK.NS': 6947327400000.0, 'ICICIBANK.NS': 2561438346000.0, 'AXISBANK.NS': 1570235200000.0, 'INDUSINDBK.NS': 632075634000.0}


In [143]:
print(debt_to_equity)

{'KOTAKBANK.NS': 0.5778612727125423, 'HDFCBANK.NS': 1.164987862814699, 'ICICIBANK.NS': 0.7847103000307781, 'AXISBANK.NS': 1.4532826037780837, 'INDUSINDBK.NS': 0.753254970432858}


In [144]:
#Calculating log returns
returns = {}
for t in tickers:
    adj_close = data[('Adj Close',t)]
    returns[t] = np.log(adj_close / adj_close.shift(1))

returns_df = pd.DataFrame(returns)
returns_df.dropna(inplace=True)
print(returns_df)

              ^BSESN  KOTAKBANK.NS  HDFCBANK.NS  ICICIBANK.NS  AXISBANK.NS  \
Date                                                                         
2021-04-01 -0.014789     -0.002399    -0.056003      0.031121     0.024712   
2021-05-01  0.062671      0.033126     0.070757      0.098635     0.048863   
2021-06-01  0.010444     -0.057992    -0.011912     -0.049250    -0.003202   
2021-07-01  0.001982     -0.030383    -0.044557      0.078615    -0.054089   
2021-08-01  0.090230      0.058048     0.103122      0.055092     0.103878   
2021-09-01  0.026981      0.134662     0.008532     -0.025637    -0.025693   
2021-10-01  0.003049      0.012709    -0.007615      0.134877    -0.032551   
2021-11-01 -0.038537     -0.034689    -0.058071     -0.115798    -0.123722   
2021-12-01  0.020621     -0.088296    -0.009519      0.035480     0.034331   
2022-01-01 -0.004122      0.033479     0.004249      0.063660     0.130386   
2022-02-01 -0.030930     -0.007838    -0.040837     -0.060221   

In [145]:
#Calculating Betas using Scikit-Learns Linear Regression
model=LinearRegression()
betas={}
bse=returns_df['^BSESN'].to_numpy().reshape(-1,1)
for t in tickers[2:]:
    Y=returns_df[t].to_numpy().reshape(-1,1)
    model.fit(bse,Y)
    betas[t]=(model.coef_[0][0])
print(betas)

{'HDFCBANK.NS': 0.9019662853487126, 'ICICIBANK.NS': 0.9722028848659506, 'AXISBANK.NS': 1.1061122785458852, 'INDUSINDBK.NS': 1.0136846518404503}


In [146]:
#Unlevering the comparison company betas and calculating the average
unlevered_betas={}
sum_betas=0
for t in tickers[2:]:
    unlevered_betas[t]=betas[t]/(1+(debt_to_equity[t]*(1-tax_rate)))
    sum_betas+=unlevered_betas[t]
avg_unlevered_beta=sum_betas/len(unlevered_betas.keys())

#Relevering the target company beta
levered_beta=avg_unlevered_beta*(1+(debt_to_equity['KOTAKBANK.NS']*(1-tax_rate)))

#Finding cost of equity using CAPM model
rfr=5.13209311/100
log_return=returns_df['^BSESN'].mean()*12
market_returns=np.exp(log_return)-1
cost_of_equity=rfr + levered_beta*(market_returns-rfr)
print(f"Cost of Equity: {cost_of_equity}")

Cost of Equity: 0.10576508070841482


In [147]:
#Finding ebit/interest expense
income_statement = yf.Ticker('KOTAKBANK.NS').financials
ebit=income_statement['2024-03-31']['Net Income']+income_statement['2024-03-31']['Interest Expense']+income_statement['2024-03-31']['Tax Provision']
interest_expense=income_statement['2024-03-31']['Interest Expense']
print(ebit/interest_expense)

2.06790908400568


In [148]:
#Calculating WACC
spread=3.5/100
cost_of_debt=spread+rfr
print(f"Cost of Debt : {cost_of_debt}")
market_val_debt=debt_vals['KOTAKBANK.NS']
shares=income_statement['2024-03-31']['Basic Average Shares']
stock_price=data[('Adj Close','KOTAKBANK.NS')]['2024-03-01']
market_val_equity=stock_price*shares

total_value=market_val_debt+market_val_equity
wacc=cost_of_equity*(market_val_equity/total_value)+cost_of_debt*(market_val_debt/total_value)
print(f"Wacc: {wacc}")

Cost of Debt : 0.0863209311
Wacc: 0.1023653590393031
